In [ ]:
from ctapipe.calib import CameraCalibrator
from ctapipe.image import tailcuts_clean
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay

from startracker.analysis.fitter import *
from startracker.io.io import *
from ctapipe.instrument import CameraGeometry
from ctapipe.coordinates import EngineeringCameraFrame
from ctapipe.io import read_table
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation
telescope_location = EarthLocation(lon=342.108612 * u.deg,
                                   lat=28.761389 * u.deg,
                                   height=2147 * u.m)

from traitlets.config import Config

import matplotlib.pyplot as plt

import matplotlib
from matplotlib.colors import PowerNorm
matplotlib.rcParams['figure.figsize'] = (12, 12)
matplotlib.rcParams.update({'font.size': 22, 'errorbar.capsize': 10})

import numpy as np
import pandas as pd
from itertools import cycle

Analyze the NSB level in data run 7761, compare it to existing simulation in order to estimate required NSB

In [ ]:
telescope_location = EarthLocation(lon=342.108612 * u.deg,
                                   lat=28.761389 * u.deg,
                                   height=2147 * u.m)
config_data =Config(
    {"LSTR0Corrections" : {
        "drs4_pedestal_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/drs4_pedestal.Run07759.0000.h5",
        "calibration_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/calibration_filters_52.Run07760.0000.h5",
        "drs4_time_calibration_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/time_calibration.Run05979.0000.h5"
    },
     "PointingSource" : {
        "drive_report_path" : "/fefs/aswg/data/real/monitoring/DrivePositioning/drive_log_22_04_09.txt"
    },
     "LocalPeakWindowSum":{
        "window_shift": 4,
        "window_width": 8,
        "apply_integration_correction": False
        }
    })

fname_data = '/fefs/aswg/data/real/R0/20220409/LST-1.1.Run07761.0120.fits.fz'

config_mc = Config({
    "LocalPeakWindowSum":{
        "window_shift": 4,
        "window_width": 8,
        "apply_integration_correction": True
        }
    })
fname_mc_0 = '/fefs/aswg/workspace/georgios.voutsinas/AllSky/TrainingDataset/GammaDiffuse/dec_2276/sim_telarray/node_corsika_theta_30.390_az_266.360_/output_v1.4/simtel_corsika_theta_30.390_az_266.360_run106.simtel.gz'
fname_mc_1 = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/simtel_corsika_theta_30.390_az_266.360_run106_nsb_836.simtel.gz'
fname_mc_2 = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/test_nsb2.4_trunc.simtel.gz'

In [ ]:
def plot_variance_hist(fname, config, tel_id=1, is_data=False, label=''):
    src = EventSource(fname,
                      config=config)
    calibrator = CameraCalibrator(subarray=src.subarray,
                                  image_extractor_type="LocalPeakWindowSum",
                                  config=config
                                  )
    camera_geometry = src.subarray.tel[tel_id].camera.geometry
    it = iter(src)
    i = 0
    event_variance_list = []
    while i < 1000:
        try:
            event =next(it)
            if event.r1.tel[tel_id].waveform is not None:
                calibrator(event)
                pixel_variances = np.var(event.r1.tel[tel_id].waveform, axis = 1)
                pixel_charges = event.dl1.tel[tel_id].image
                shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                             picture_thresh=30, boundary_thresh=15)
                if np.all(shower_mask == False):
                    continue
                pixel_variances[shower_mask] = np.mean(pixel_variances[~shower_mask])
                event_variance_list.append(pixel_variances)
                i = i+1
        except StopIteration:
            break
        
    if is_data:
        average_image = np.sum(event_variance_list, axis=0) / len(event_variance_list)
    else:
        average_image = np.sum(event_variance_list, axis=0) / len(event_variance_list)
    print(pd.DataFrame(average_image).describe())
    plt.hist(average_image, bins=1000, range=(0,1), alpha=0.5, label=label)
    plt.legend()

In [ ]:
# plot_variance_hist(fname_mc_0, config_mc, label='Nominal MC')
# plot_variance_hist(fname_mc_1, config_mc, label='NSB 836MHz')
# plot_variance_hist(fname_mc_2, config_mc, label='NSB 2842MHz')
# plot_variance_hist(fname_data, config_data, is_data=True, label='Data')

In [ ]:
def plot_variance_hist_nsb_injection(fname, config, tel_id=1, is_data=False, nsb_injection_factor = 1):
    src = EventSource(fname,
                      config=config)
    calibrator = CameraCalibrator(subarray=src.subarray,
                                  image_extractor_type="LocalPeakWindowSum",
                                  config=config
                                  )
    camera_geometry = src.subarray.tel[tel_id].camera.geometry
    it = iter(src)
    i = 0
    event_variance_list = []
    while i < 500:
        try:
            event =next(it)
            wf = event.r1.tel[tel_id].waveform
            if event.r1.tel[tel_id].waveform is not None:
                calibrator(event)
                if not is_data:
                    wf = wf * nsb_injection_factor
                pixel_variances = np.var(wf, axis = 1)
                pixel_charges = event.dl1.tel[tel_id].image
                shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                             picture_thresh=30, boundary_thresh=15)
                if np.all(shower_mask == False):
                    continue
                pixel_variances[shower_mask] = np.mean(pixel_variances[~shower_mask])
                event_variance_list.append(pixel_variances)
                i = i+1
        except StopIteration:
            break
    average_image = np.sum(event_variance_list, axis=0) / len(event_variance_list)
    print(pd.DataFrame(average_image).describe())
    plt.hist(average_image, bins=1000, range=(0,1), alpha=0.5)

In [ ]:
def produce_cumulative_waveforms(fname, config):
    src = EventSource(fname, config=config)
    calibrator = CameraCalibrator(subarray=src.subarray,
                                  image_extractor_type="LocalPeakWindowSum",
                                  config=config
                                  )
    camera_geometry = src.subarray.tel[1].camera.geometry
    r0 = np.array([])
    r1 = np.array([])
    c = np.array([])
    i = 0
    for event in src:
        try:
            calibrator(event)
            pixel_charges = event.dl1.tel[1].image
            if pixel_charges is not None:
                shower_mask = tailcuts_clean(camera_geometry,
                                             pixel_charges,
                                             picture_thresh=30, boundary_thresh=15)
                r0 = np.append(r0, event.r0.tel[1].waveform[0,~shower_mask,:].flatten())
                r1 = np.append(r1, event.r1.tel[1].waveform[~shower_mask,:].flatten())
                c = np.append(c, pixel_charges[~shower_mask])
                i = i + 1
        except StopIteration:
            break
        if i > 1000:
            break
    return r0, r1, c

In [ ]:
# r0_data, r1_data, c_data = produce_cumulative_waveforms(fname_data, config_data)
# r0_mc_0, r1_mc_0, c_mc_0 = produce_cumulative_waveforms(fname_mc_0, config_mc)
# r0_mc_1, r1_mc_1, c_mc_1 = produce_cumulative_waveforms(fname_mc_1, config_mc)
# r0_mc_2, r1_mc_2, c_mc_2 = produce_cumulative_waveforms(fname_mc_2, config_mc)

In [ ]:
# fig, ax = plt.subplots()
# t=ax.hist(c_data, bins=1000, range=(0,100), alpha=0.3, label='Data')
# t=ax.hist(c_mc_0, bins=1000, range=(0,100), alpha=0.3)
# t=ax.hist(c_mc_1, bins=1000, range=(0,100), alpha=0.3)
# t=ax.hist(c_mc_2, bins=1000, range=(0,100), alpha=0.3)

In [ ]:
def plot_image(filename, telescope_location, index=0):
    image = []
    star_pixels = []
    expected_x = []
    expected_y = []
    reco_x = []
    reco_y = []
    stars = {}
    
    for key in get_dataset_keys(filename):
        if 'variance_images' in key:
            image_table = read_table(filename, f'/{key}')
            image = image_table['image'][index]
            t = Time(image_table['timestamp'][index], format='unix_tai', scale='utc')
            pointing = SkyCoord(image_table['ra'][index],
                                image_table['dec'][index],
                                unit='deg', frame='icrs', obstime=t,
                                location=telescope_location)
            pointing.transform_to('altaz')
        else:
            star_table = read_table(filename, f'/{key}')
#             cluster = star_table['pixels'][index]
#             cluster = cluster[cluster!= -1]
#             star_pixels.extend(cluster)
            if not np.isnan(star_table['reco_x'][index]): 
                cluster = star_table['pixels'][index]
                cluster = cluster[cluster!= -1]
                star_pixels.extend(cluster)
                stars[key] = cluster
                print(f'Expected star coordinates:\n'
                      f'x={-star_table["expected_x"][index]},\n'
                      f'y={-star_table["expected_y"][index]},\n'
                      f'Cluster: {cluster}'
                      )
            #if not np.isnan(star_table['reco_x'][index]): 
                expected_x.append(-star_table['expected_x'][index])
                expected_y.append(-star_table['expected_y'][index])
                reco_x.append(-star_table['reco_x'][index])
                reco_y.append(-star_table['reco_y'][index])
    frame = EngineeringCameraFrame(telescope_pointing=pointing,
                               focal_length=28*u.m,
                               obstime=t.utc,
                               location=telescope_location)
    fig, ax = plt.subplots()
    
    geom = CameraGeometry.from_name('LSTCam-003')
    neighbors = []
    for pix in star_pixels:
        neighbors.extend(geom.neighbors[pix])
    
    star_pixels.extend(neighbors)
    star_pixels = np.unique(star_pixels)
    star_mask = np.ones(image.size, dtype=bool)
    star_mask[star_pixels] = False
    nsb = np.mean(image[star_mask])
    print(f'NSB level: {nsb}')
    print(f'NSB stats: \n{pd.DataFrame(image[star_mask]).describe()}')
    
    image[~star_mask] = nsb

    geom.frame=frame
    disp = CameraDisplay(geom, image, ax)
    disp.norm = PowerNorm(gamma=0.3)
    disp.highlight_pixels(star_pixels, 'red', linewidth=2)
    disp.add_colorbar()
    print(f'Star pixels total {star_pixels}')
    ax.grid(True)
    return star_pixels, stars

In [ ]:
nsb_mask, stars = plot_image('/fefs/aswg/workspace/mykhailo.dalchenko/pointing/data/run_7761_v5/out/run_7761.244.h5',telescope_location, 5)

In [ ]:
nsb_mask.shape

In [ ]:
stars

In [ ]:
stars.pop('startracker/images/stars/S_1158_0196865')

In [ ]:
data_reference = '/fefs/aswg/data/real/R0/20220409/LST-1.1.Run07761.0244.fits.fz'
src = EventSource(data_reference,
                  config=config_data)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_data
                              )

variances_nsb = []
charges_nsb = []
variance_stars = {k: [] for k in stars.keys()}
average_variance_stars = {k: [] for k in stars.keys()}
std_variance_stars = {k: [] for k in stars.keys()}

average_variance_nsb = []

camera_geometry = src.subarray.tel[1].camera.geometry

for i in range(1, 300):
    with EventSource(data_reference, config=config_data) as t_src:
        variance_stars = {k: [] for k in stars.keys()}
        for j in range(0,i):
            event = next(iter(t_src))
            calibrator(event)
            pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
            #print(pixel_variances.shape)
            pixel_charges = event.dl1.tel[1].image
            shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                         picture_thresh=30, boundary_thresh=15)
            #print(f'Shower mask: {shower_mask}')
            safe_mask = shower_mask.copy()
            safe_mask[nsb_mask] = True
            #print(f'Safe mask: {safe_mask}')
            #print(pixel_variances[~safe_mask].shape)
            variances_nsb.extend(pixel_variances[~safe_mask])
            #variances_nsb.append(np.average(pixel_variances[~safe_mask]))
            pixel_variances[shower_mask] = np.mean(pixel_variances[~safe_mask])


            #charges_nsb.extend(pixel_charges[~safe_mask])
            for k in variance_stars.keys():
        #         print(f'Star cluster {stars[k]}')
        #         print(f'shower mask {np.where(shower_mask == True)}')
#                 if np.isin(stars[k], np.where(shower_mask == True)).any():
#                     variance_stars[k].append(np.nan)
                sv = np.sum(pixel_variances[stars[k]])
                variance_stars[k].append(sv)
        for k in variance_stars.keys():
            average_variance_stars[k].append(np.nanmean(variance_stars[k]))
            std_variance_stars[k].append(np.nanstd(variance_stars[k]))
        average_variance_nsb.append(np.nanmean(variances_nsb))

In [ ]:
fig, ax = plt.subplots()
lines = ["--","-.",":", (0, (5, 1)), "-"]
linecycler = cycle(lines)
x = range(1,300)
for k in variance_stars.keys():
    label = k.split('/')[-1][2:].replace('_','-')
    #ax.errorbar(x, average_variance_stars[k], yerr=std_variance_stars[k], label=f'Average variance of star {label}')
    ax.plot(x, average_variance_stars[k],
            linestyle=next(linecycler),
            linewidth=3,
            label=f'Star {label}')
    
ax.plot(x, average_variance_nsb,
        linestyle=next(linecycler),
        linewidth=3,
        label="Average NSB level")
    
#ax.grid()
ax.legend()
ax.set_ylim(0, 8)
ax.set_ylabel(r'Average variance of a star $[p.e.^2]$')
ax.set_xlabel('Number of events used to produce average variance')
plt.savefig('/fefs/aswg/workspace/mykhailo.dalchenko/variance_stability.pdf', format='pdf', dpi=300, bbox_inches = "tight")

In [ ]:
data_reference = '/fefs/aswg/data/real/R0/20220409/LST-1.1.Run07761.0244.fits.fz'
src = EventSource(data_reference,
                  config=config_data)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_data
                              )

variances_nsb = []
charges_nsb = []
variance_stars = {k: [] for k in stars.keys()}

camera_geometry = src.subarray.tel[1].camera.geometry
i = 0
for event in src:
    calibrator(event)
    pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
    #print(pixel_variances.shape)
    pixel_charges = event.dl1.tel[1].image
    shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                 picture_thresh=30, boundary_thresh=15)
    #print(f'Shower mask: {shower_mask}')
    safe_mask = shower_mask.copy()
    safe_mask[nsb_mask] = True
    #print(f'Safe mask: {safe_mask}')
    #print(pixel_variances[~safe_mask].shape)
    variances_nsb.extend(pixel_variances[~safe_mask])
    #variances_nsb.append(np.average(pixel_variances[~safe_mask]))


    charges_nsb.extend(pixel_charges[~safe_mask])
    for k in variance_stars.keys():
#         print(f'Star cluster {stars[k]}')
#         print(f'shower mask {np.where(shower_mask == True)}')
        if np.isin(stars[k], np.where(shower_mask == True)).any():
            continue
        sv = np.sum(pixel_variances[stars[k]])
        variance_stars[k].append(sv)
#     i = i + 1
#     if i > 10000:
#         break


In [ ]:
nsb, bins, patches = plt.hist(variances_nsb, bins=100, range=(0,10), density=True, label='NSB')
variance_hist_stars = {k: None for k in stars.keys()}
for k, v in variance_stars.items():
#     print('----------------------------------------')
#     print(v)
#     print('----------------------------------------')
    n, bins, patches = plt.hist(v, bins=100, histtype=u'step', linewidth=3, label=k, range=(0,10), density=True)
    variance_hist_stars[k] = n
#plt.yscale('log')
plt.legend()
plt.xlabel(r'Variance $[p.e.^2]$')
plt.ylabel('a.u.')
plt.grid(True)

In [ ]:
for k, v in variance_stars.items():
    print('----------------------------------------')
    print(v[:10])
    print('----------------------------------------')
    plt.hist(v, bins=100, histtype=u'step', linewidth=2, label=k, range=(0,20))
plt.yscale('log')
plt.legend()

In [ ]:
plt.step((bins[1:] + bins[:-1])/2, nsb)
plt.xlim(0,2)

In [ ]:
bin_centers = (bins[1:] + bins[:-1])/2
nsb_av = np.average(bin_centers, weights=nsb)

In [ ]:
nsb_av

In [ ]:
for k,v in variance_hist_stars.items():
    print(f'Average variance for star {k}:\n{np.average(bin_centers, weights=v)}')

In [ ]:
sim_reference = fname_mc_0
src = EventSource(sim_reference,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )

variances_nsb_sim = []


camera_geometry = src.subarray.tel[1].camera.geometry
i = 0
for event in src:
    calibrator(event)
    if event.r1.tel[1].waveform is not None:
        pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
        #print(pixel_variances.shape)
        pixel_charges = event.dl1.tel[1].image
        shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                     picture_thresh=30, boundary_thresh=15)

        variances_nsb_sim.extend(pixel_variances[~shower_mask])

        i = i + 1
        if i > 1000:
            break



In [ ]:
nsb_sim, bins_sim, _ = plt.hist(variances_nsb_sim, bins=100, range=(0,1), density=True, label='NSB Simulation')

In [ ]:
bin_centers_sim = (bins_sim[1:] + bins_sim[:-1])/2
nsb_sim_av = np.average(bin_centers_sim, weights=nsb_sim)

In [ ]:
nsb_sim_av

In [ ]:
0.48211155714816123/0.037680045925003694

In [ ]:
0.48211155714816123/0.037680045925003694 * 0.24586

In [ ]:
sim_reference = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/test_nsb_3.8GHz-HT.simtel.gz'
src = EventSource(sim_reference,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )

variances_nsb_sim = []


camera_geometry = src.subarray.tel[1].camera.geometry
i = 0
for event in src:
    try:
        calibrator(event)
        if event.r1.tel[1].waveform is not None:
            pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
            #print(pixel_variances.shape)
            pixel_charges = event.dl1.tel[1].image
            shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                         picture_thresh=30, boundary_thresh=15)

            variances_nsb_sim.extend(pixel_variances[~shower_mask])

            i = i + 1
            if i > 1000:
                break
    except Exception as e:
        print(i)



In [ ]:
nsb_sim, bins_sim, _ = plt.hist(variances_nsb_sim, bins=100, range=(0,10), density=True, label='NSB Simulation')

In [ ]:
bin_centers_sim = (bins_sim[1:] + bins_sim[:-1])/2
nsb_sim_av = np.average(bin_centers_sim, weights=nsb_sim)

In [ ]:
nsb_sim_av

In [ ]:
plt.step((bins[1:] + bins[:-1])/2, nsb, label = 'Data')
plt.step((bins[1:] + bins[:-1])/2, nsb_sim)
plt.legend()
plt.xlim(0,2)

In [ ]:
3.8*(3.75037522901183-0.48376160336586865)/0.48376160336586865

In [ ]:
3.8*(1.4362496770464468-0.48376160336586865)/0.48376160336586865

In [ ]:
3.8*(4.487377626323164-0.48376160336586865)/0.48376160336586865

In [ ]:
3.8*(5.66156369031502-0.48376160336586865)/0.48376160336586865

In [ ]:
def read_star_lis(fname):
    colnames = ['PixelN', 'X', 'Y', 'Star']
    df = pd.read_csv(fname, sep=' ', usecols=[1,2,3,15], skiprows=35, names=colnames)
    df['X'] = df['X']/100.0
    df['Y'] = df['Y']/100.0
    df['Star'] = df['Star']
    df = df[df['PixelN'] != -1]
    return df

In [ ]:
star_pixels_df = read_star_lis('/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/star_photons_hstat2_realistic.lis')

In [ ]:
labels = [int(i) for i in range(1, 5)]
simulated_stars = {}
for l in labels:
    simulated_stars[l] = np.unique(star_pixels_df[star_pixels_df['Star'] == l]['PixelN'])

In [ ]:
simulated_stars

In [ ]:
nsb_mask_sim = np.concatenate([v for k, v in simulated_stars.items()])
nsb_mask_sim

In [ ]:
simulated_stars[4] = np.array([0])

In [ ]:
sim_reference = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/test_nsb_3.8GHz-HT_stars_high_stat2_realistic.simtel.gz'
src = EventSource(sim_reference,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )

variances_nsb_sim = []
variance_stars_sim = {k: [] for k in simulated_stars.keys()}


cluster_size_correction = {1:4.36, 2:4.36, 3:1.45, 4:0}

camera_geometry = src.subarray.tel[1].camera.geometry
i = 0
for event in src:
    calibrator(event)
    if event.r1.tel[1].waveform is not None:
        pixel_variances = np.var(event.r1.tel[1].waveform, axis = 1)
        #print(pixel_variances.shape)
        pixel_charges = event.dl1.tel[1].image
        shower_mask = tailcuts_clean(camera_geometry, pixel_charges,
                                     picture_thresh=30, boundary_thresh=15)

        safe_mask = shower_mask.copy()
        safe_mask[nsb_mask_sim] = True
        #print(f'Safe mask: {safe_mask}')
        #print(pixel_variances[~safe_mask].shape)
        variances_nsb_sim.extend(pixel_variances[~safe_mask])
        #variances_nsb.append(np.average(pixel_variances[~safe_mask]))
        for k in variance_stars_sim.keys():
#             print(f'Star cluster {simulated_stars[k]}')
#             print(f'shower mask {np.where(shower_mask == True)}')
            if np.isin(simulated_stars[k], np.where(shower_mask == True)).any():
                continue
            sv = np.sum(pixel_variances[simulated_stars[k]]) - cluster_size_correction[k]
            variance_stars_sim[k].append(sv)
        i = i + 1
#         if i > 1000:
#             break



In [ ]:
nsb_sim, bins_sim, patches = plt.hist(variances_nsb_sim, bins=100, range=(0,10), density=True, label='NSB')
variance_hist_stars_sim = {k: None for k in simulated_stars.keys()}
for k, v in variance_stars_sim.items():
#     print('----------------------------------------')
#     print(v)
#     print('----------------------------------------')
    n, bins, patches = plt.hist(v, bins=100, histtype=u'step', linewidth=3, label=k, range=(0,10), density=True)
    variance_hist_stars_sim[k] = n
#plt.yscale('log')
plt.legend()
plt.xlabel('Variance [pe]')
plt.ylabel('a.u.')
plt.grid(True)

In [ ]:
for k, v in variance_stars_sim.items():
#     print('----------------------------------------')
#     print(v)
#     print('----------------------------------------')
    n, bins, patches = plt.hist(v, bins=100, histtype=u'step', linewidth=3, label=k, range=(0,10), density=True)
for k, v in variance_stars.items():
#     print('----------------------------------------')
#     print(v)
#     print('----------------------------------------')
    n, bins, patches = plt.hist(v, bins=100, histtype=u'step', linewidth=3, label=k, range=(0,10), density=True)
plt.legend()
plt.grid()

In [ ]:
disp = CameraDisplay(camera_geometry)
disp.norm = PowerNorm(gamma=0.3)
disp.highlight_pixels(simulated_stars[3], color='r')
disp.add_colorbar()

In [ ]:
for k,v in variance_hist_stars_sim.items():
    print(f'Average variance for star {k}:\n{np.average(bin_centers, weights=v)}')

In [ ]:
print(f'Average variance for nsb:\n{np.average(bin_centers, weights=nsb_sim)}')

In [ ]:
for k, v in simulated_stars.items():
    print(f'{k}: cluster length {v.shape}')

In [ ]:
1.4362496770464468 - 0.482

In [ ]:
0.7688118684158723 - 0.4838989651150047

In [ ]:
0.9542496770464468/0.28491290330086766

In [ ]:
for k, v in stars.items():
    print(f'{k}: cluster length {v.shape}')
    print(f'residual variance from star is {np.average(bin_centers, weights=variance_hist_stars[k]) - v.shape[0] * 0.482}')

In [ ]:
5 * 3.34

In [ ]:
1.8055636903150205/0.9542496770464468 * 16.7

In [ ]:
3 * 0.4838989651150047

In [ ]:
test_file = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/test_nsb_3.145GHz.simtel.gz'

In [ ]:
src = EventSource(test_file,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )
i = 0
for event in src:
    calibrator(event)
    print(event.trigger.time.to_value('ymdhms'))
    i = i+1
    if i > 9:
        break
    

In [ ]:
src = EventSource(test_file,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )
i = 0
for event in src:
    calibrator(event)
    print(f'Initial time {event.trigger.time}')
    event.trigger.time = i
    print(f'Modified time {event.trigger.time}')
    i = i+1
    if i > 9:
        break

In [ ]:
start_t = 1649540000
#stop_t = 1649546000
stop_t = 1649540003
dt = stop_t - start_t
print(dt*10)

In [ ]:
times = np.linspace(start_t, stop_t, (stop_t-start_t)*10+1)
len(times)

In [ ]:
sim_reference = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/test/simulation_startracker/subrun_1/output//1649540000.0.simtel.gz'
src = EventSource(sim_reference,
                  config=config_mc)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_mc
                              )

variances_nsb_sim = []


camera_geometry = src.subarray.tel[1].camera.geometry
i = 0
event = next(iter(src))
calibrator(event)
image = np.var(event.r1.tel[1].waveform, axis = 1)
disp = CameraDisplay(camera_geometry)
disp.norm = PowerNorm(gamma=0.5)
disp.image = image
disp.add_colorbar()

In [ ]:
data_reference = '/fefs/aswg/data/real/R0/20220409/LST-1.1.Run07761.0060.fits.fz'
src = EventSource(data_reference,
                  config=config_data)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_data
                              )

In [ ]:
it = iter(src)
event = next(it)
src.r0_r1_calibrator.calibrate(event)

In [ ]:
event.mon.tel[1].pixel_status.hardware_failing_pixels.shape

In [ ]:
event.mon.tel[1].pixel_status

In [ ]:
np.argwhere(event.mon.tel[1].pixel_status.hardware_failing_pixels == True)

In [ ]:
np.argwhere(event.mon.tel[1].pixel_status.pedestal_failing_pixels == True)

In [ ]:
np.argwhere(event.mon.tel[1].pixel_status.flatfield_failing_pixels == True)

In [ ]:
event.mon.tel[1].calibration.unusable_pixels

In [ ]:
event.trigger.event_type

In [ ]:
calibrator(event)

In [ ]:
event.mon.tel[1].calibration.unusable_pixels

In [ ]:
event.calibration.tel[1].dl1

In [ ]:
import ctapipe

In [ ]:
print(ctapipe.__version__)